# Delta Project Controller

### Packages

In [2]:
from stupidArtnet import StupidArtnet
import copy
import random
import string
import sys
import time
import os
import PySimpleGUI as sg
import pygame_gui
import pygame
from pygame.event import Event
from pygame_gui.elements import UIButton
from pygame_gui.windows import UIColourPickerDialog
from pynput.mouse import Listener
import pyautogui
import json
from PIL import Image, ImageGrab

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


### Constants

In [3]:
# Setup Constants
DEFAULT_PACKET_SIZE = 512
DEFAULT_FPS = 40
ENFORCE_EVEN_PACKET = True
ENFORCE_BROADCAST = True
DEFAULT_UNIVERSE_ID = 1
DEFAULT_CHANNEL_START_ID = 1
DEFAULT_CHANNEL_WIDTH = 11
PRESETS_PATH = '../presets/preset.json'
BITFOCUS_CONFIG_FOLDER = '../config/'
BITFOCUS_CONFIG_PATH = BITFOCUS_CONFIG_FOLDER+'bitfocus_config.json'
DEFAULT_IP = '169.254.79.148'
PRESET_BUTTON_PREFIX = 'preset_group_'
# Control Constants
DIMMER_ID = 1
DIMMER_FINE_ID = 2
STROBE_ID = 3
RED_ID = 4
GREEN_ID = 5
BLUE_ID = 6
WHITE_ID = 7
AMBER_ID = 8
UV_ID = 9
PRESET_ID = 10
SOUND_ID = 11
DEFAULT_LIGHT_VALUE = [255]+[0]*(DEFAULT_CHANNEL_WIDTH-1)
RESET_VALUE = [0]*DEFAULT_CHANNEL_WIDTH
LIGHT_OFF_VALUE = RESET_VALUE
# Config
MAX_BUTTON_ID = 32
DEFAULT_FADE_TIME = 750
DEFAULT_ID_LENGTH = 21
# Mappings
FIXTURE_TO_ID_DICT = {'dimmer':DIMMER_ID,'red':RED_ID,'green':GREEN_ID,
                    'blue':BLUE_ID,'white':WHITE_ID,'amber':AMBER_ID,
                    'uv':UV_ID}
ID_TO_FIXTURE_DICT = dict([(i,fixture) for fixture,i in FIXTURE_TO_ID_DICT.items()])
DEFAULT_GROUPS = {'group_1':['light_1','light_3','light_5'],
                  'group_2':['light_2','light_4','light_6']}
# Events
LIGHT_SELECTION_EVENTS = {'group_1','group_2','light_1','light_2','light_3',
              'light_4','light_5','light_6'}
LIGHT_FIXTURE_EVENTS = set(['slider_'+fixture for fixture in FIXTURE_TO_ID_DICT.keys()])

### Code

#### Objects

##### Channel

In [4]:
class Channel:
    """
    ArtNet Channel Object. A Channel is to a range of
    fixture of size channel_width. It is modeled by a
    simple part of the underlying artnet server. 
    
    """
    def __init__(self,server:StupidArtnet,channel_start:int,channel_width:int):
        """
        Instantiate an ArtNet Channel object.

        :param server: ArtNet server to communicate with the lights.
        :param channel_start: Id of the starting DMX address.
        :param channel_width: Width of the channel, equal to fixture number.
        
        """
        self.server = server
        self.channel_start = channel_start
        self.channel_width = channel_width
        self.offset = self.channel_start - 1

    def set_value(self,fixture_id:int,value:int,show=True):
        """
        Set the given fixture to the provided value.

        :param fixture_id: Id of the DMX fixture.
        :param value: Value of the fixture. Should be contained in [0,255].
        :param show: If set to True, then the packet will be sent.
        
        """
        if value < 0 or value > 255:
            raise ValueError(f'The value for {ID_TO_FIXTURE_DICT[fixture_id]} should be contained in [0,255]')
        self.server.set_single_value(self.offset+fixture_id,value)
        time.sleep(0.002)
        if show:
            self.server.show()
        time.sleep(0.002)

    def set_values(self,values:list):
        """
        Set all the fixtures of the channel to the given list of values.

        :param values: List of the fixtures values.
        
        """
        if len(values) != self.channel_width:
            raise ValueError(f'The list of values sent by the channel must be of size equal to the channel width: {self.channel_width}')
        for fixture_id, value in enumerate(values):
            self.set_value(fixture_id+1, value, show=False)

    def set_values_(self,values:list):
        """
        Set all the fixtures of the channel to the given list of values.

        :param values: List of the fixtures values.
        
        """
        self.set_multiple_values(self.offset+1,self.offset+self.channel_width,values)
    
    def reset(self):
        """ Reset the channel to its default state. """
        self.set_values(RESET_VALUE)

##### Light Source

In [5]:
class LightSource:
    """ Abstract Light Source Object """
    
    def __init__(self,name:str,state=DEFAULT_LIGHT_VALUE.copy()):
        """ Instantiate the Light Source, defined by a name and a state. """
        self.name = name
        self.state = state

    def set_fixture_value(self,fixture_id:int, value:int):
        """ Define the value of a fixture. """

    def set_fixture_values(self,fixture_id:int, value:int):
        """ Define the values of all fixtures. """

    def blink():
        """ Make the Light Source Blink. """

    def set_rgb(self, values:list):
        """ Define the RGB values. """

    def turn_off(self):
        """ Turn off the Light Source"""

    def turn_on(self):
        """ Turn on the Light Source"""

    def reset(self):
        """ Reset the Light Source to its default state. """

###### Light

In [6]:
class Light(LightSource):
    """
    Light object class. A light is model by the channel to which
    it is linked and a state. The latter is a list containing the 
    values for each of the fixture in the channel. 
    
    """
    
    def __init__(self,name:str, channel:Channel):
        """
        Create a Light instance.

        :param name: String identifier for the light. If the light is used in a Group,
                     please be sure to enter unique identifiers.
        :param channel: ArtNet channel object to which the light is bound. 

        """
        super().__init__(name)
        self.group_name = ''
        self.channel = channel
        self.turn_on()
        
    def set_fixture_value(self,fixture_id:int,value:int):
        """
        Set the fixture to the given value.

        :param fixture_id: Id of the fixture to be set.
        :param value: Integer value of the fixture, should be contained in [0,255].
        
        """
        new_state = self.state.copy()
        new_state[fixture_id-1] = value
        self.channel.set_value(fixture_id, value)
        self.state = new_state

    def set_fixture_values(self,values=[]):
        """
        Set the fixtures to the given list of values.

        :param values: Ordered list of values to be set.
        
        """
        self.channel.set_values(values)
        new_state = values
        self.state = new_state

    def set_rgb(self, values:list):
        """
        Set the RGB fixtures to the color code given in values.

        :param values: List containing the values for the red, green and blue fixtures.
        
        """
        for idx, fixture_id in enumerate([RED_ID,GREEN_ID,BLUE_ID]):
            self.set_fixture_value(fixture_id,values[idx])

    def blink(self,blink_time=0.2,n_repeat=2):
        """ """
        prev_state = self.state.copy()
        self.turn_off()
        time.sleep(blink_time)
        self.reset()
        self.set_fixture_value(WHITE_ID, 255)
        for i in range(n_repeat):
            time.sleep(blink_time)
            self.turn_on()
            time.sleep(blink_time)
            self.turn_off()
        self.set_fixture_values(prev_state)
        self.turn_on()

    def turn_off(self):
        """ Turn off the light by setting dimmer to 0. """
        self.set_fixture_value(DIMMER_ID,0)

    def turn_on(self):
        """ Turn on the light by setting dimmer to 255. """
        self.set_fixture_value(DIMMER_ID,255)

    def reset(self):
        """ Reset the light to its default state, i.e. zero value for each fixture. """
        self.turn_off()
        self.channel.reset()
        new_state = RESET_VALUE
        self.state = new_state

###### Group

In [7]:
class Group(LightSource):
    """
    Group object class. A group is modeled by a list
    of lights. Every action applied to the group
    will be executed on all lights present. Each light in the 
    group should be unique and have a unique name.
    
    """
    
    def __init__(self,name:str, lights=[]):
        """
        Create a Group instance.

        :param name: String identifier for the group of lights.
        :param lights: List containing the initial lights of the group,
                       empty by default. All lights should be unique.
        
        """
        super().__init__(name)
        self.lights = []
        self.light_names = set()
        if len(self.light_names) != len(self.lights):
            raise ValueError('Duplicate names in the list of lights provided to the Group constructor')
        for l in lights:
            self.add_light(l)

    def add_light(self,light:Light):
        """
        Add a light to the existing pool. Must be a new unique light.

        :param light. Light object to add to the existing pool of lights in the group.
        
        """
        if light.name in self.light_names:
            raise ValueError('Tried to add a light whose name is already present in the group.')
        if light.group_name != '' and light.group_name != self.name:
            raise ValueError('Tried to add a light which is already present in another group.') 
        self.lights.append(light)
        light.set_fixture_values(self.state)
        light.group_name = self.name
        self.light_names = self.light_names.union(set(light.name))

    def remove_light(self,light_name:str):
        """
        Remove a light from the existing pool.

        :param light_name: Identifier of the light to be removed.
        
        """
        light.group_name = ''
        self.light_names = self.light_names.difference()
        self.lights = [l for l in self.lights if l.name != light_name]
        self.light.reset()
        
        
    def set_fixture_value(self, fixture_id:int, value:int):
        """
        Set the given fixture to 'value' for all lights in the group.

        :param fixture_id: Id of the fixture to be set.
        :param value: Integer value of the fixture, should be contained in [0,255].
        
        """
        new_state = self.state.copy()
        for l in self.lights:
            l.set_fixture_value(fixture_id,value)
        new_state[fixture_id-1] = value
        self.state = new_state

    def set_fixture_values(self,values=[]):
        """
        Set the fixtures to the given list of values for all lights in the group.

        :param values: Ordered list of values to be set.
        
        """
        for l in self.lights:
            l.set_fixture_values(values)
        new_state = values
        self.state = new_state

    def set_rgb(self, values:list):
        """
        Set the RGB fixtures to the color code given in values.

        :param values: List containing the values for the red, green and blue fixtures.
        
        """
        for idx, fixture_id in enumerate([RED_ID,GREEN_ID,BLUE_ID]):
            self.set_fixture_value(fixture_id,values[idx])

    def blink(self):
        """ Make all lights in the group blink. """
        for l in self.lights:
            l.blink()

    def turn_off(self):
        """ Turn off the light by setting dimmer to 0. """
        self.set_fixture_value(DIMMER_ID,0)

    def turn_on(self):
        """ Turn on the light by setting dimmer to 255. """
        self.set_fixture_value(DIMMER_ID,255)

    def reset(self):
        """ Reset all lights in the group to their default states, i.e. zero value for each fixture. """
        for l in self.lights:
            l.reset()

#### Presets

In [8]:
presets = {'base' : 
            {'base' : {'group_1':[255,0,0,255,180,50,50,255,0,0,0],'group_2':[255,0,0,255,180,50,50,255,0,0,0],
                'light_1':[255,0,0,255,180,50,50,255,0,0,0],'light_2':[255,0,0,255,180,50,50,255,0,0,0],
                'light_3':[255,0,0,255,180,50,50,255,0,0,0],'light_4':[255,0,0,255,180,50,50,255,0,0,0],
                'light_5':[255,0,0,255,180,50,50,255,0,0,0],'light_6':[255,0,0,255,180,50,50,255,0,0,0]},
             'violet' : {'group_1':[255,0,0,255,0,255,0,0,0,0,0],'group_2':[255,0,0,255,0,255,0,0,0,0,0],
                'light_1':[255,0,0,255,0,255,0,0,0,0,0],'light_2':[255,0,0,255,0,255,0,0,0,0,0],
                'light_3':[255,0,0,255,0,255,0,0,0,0,0],'light_4':[255,0,0,255,0,255,0,0,0,0,0],
                'light_5':[255,0,0,255,0,255,0,0,0,0,0],'light_6':[255,0,0,255,0,255,0,0,0,0,0]}}}


In [9]:
#with open(PRESETS_PATH,'w') as file:
#    json.dump(presets,file)

#### Config

In [10]:
def create_config_structure(ip_address:str,preset:dict,number_of_pages:int=10,
                            instance_id:str="TKdJlb-N6u8sGy0ufAlx1") -> dict:
    """ 
    Create global BitFocus Companion config skeleton. 
    
    :param ip_address: IP address of the artnet module.
    :param number_of_pages: Number of button pages to generate.

    :return: Config in dictionnary format.
    """
    config = {'version':3,'type':'full','pages':dict([(str(i+1),{"name":"PAGE"}) for i in range(number_of_pages)]),
              'controls':create_controls_config(preset,instance_id),
              'instances':{instance_id:{"instance_type":"generic-artnet",
                        "sortOrder":1,"label":"artnet","isFirstInit":False,
                        "config":{"host":ip_address,"universe":1,
                        "timer_slow":1000,"timer_fast":40},"enabled":True,
                        "lastUpgradeIndex":0}}}
    return config

In [11]:
def generate_config_id(id_length:int) -> str:
    """ Generate ascii random id of given length. """
    return ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=id_length))

In [12]:
def create_channel_config(channel_id:int,channel_value:int,
                            instance_id:str,id_length:int,
                            fade_time:int) -> dict:
    """ Create config step for a single channel. """
    channel_dict = {"id":generate_config_id(id_length),"action":"set",
                    "instance":instance_id,
                    "options":{"channel":channel_id,"value":channel_value,
                    "duration":fade_time},"delay":0}
    return channel_dict

In [13]:
def create_button_config(preset_config:dict,instance_id:str,preset_name:str,
                        id_length:int,fade_time:int) -> dict:
    """ """
    light_values = [v for values in list(preset_config.values())[2:] for v in values]
    light_data = [(idx+1,light_values[idx]) for idx in range(len(light_values))]
    button_config = {"type":"button",
                     "style":{"text":preset_name,"size":"auto","png":None,
                              "alignment":"center:top","pngalignment":"center:center",
                              "color":16777215,"bgcolor":0,"show_topbar":True},
                     "options":{"relativeDelay":False,"stepAutoProgress":True},
                     "feedbacks":[],
                     "steps":{"0":{"action_sets":{"down":[
                         create_channel_config(idx,value,instance_id,id_length,fade_time) for
                         idx,value in light_data],
                     "up":[]},"options":{"runWhileHeld":[]}}}}
    return button_config
    

In [14]:
def create_controls_config(preset:dict,instance_id:str,id_length:int=DEFAULT_ID_LENGTH,
                            fade_time:int=DEFAULT_FADE_TIME) -> dict:
    """ """
    control_dict = dict()
    bank_id = 1
    button_id = 1
    for preset_name, preset_config in preset.items():
        button_config = create_button_config(preset_config,instance_id,
                                                preset_name,id_length,fade_time)
        control_dict[f'bank:{bank_id}-{button_id}'] = copy.deepcopy(button_config)
        # If we reached the end of the page
        if button_id == MAX_BUTTON_ID:
            bank_id +=1
            button_id = 0
        button_id += 1
    return control_dict


In [15]:
with open(PRESETS_PATH) as f:
    a = json.load(f)

In [16]:
b = create_config_structure(DEFAULT_IP,a['base'])

In [17]:
with open(BITFOCUS_CONFIG_PATH,'w') as f:
    json.dump(b,f)

#### GUI

##### Layout

In [18]:
def create_preset_layout(presets:dict) -> list:
    """ Create the layout for the preset menu. """
    layout = []
    for preset_group in presets.keys():
        layout.append([sg.Button(preset_group,button_color="black on SkyBlue1",key=PRESET_BUTTON_PREFIX+preset_group,s=(90,5))])
    return layout

In [19]:
def create_preset_selector_layout(presets:dict,preset_group:str) -> list:
    """ Create the layout of the popup window for presets. """
    layout = [[sg.Text(preset_group,justification='center',font='bold',s=(80,3))]]
    for preset_name, preset_state in presets[preset_group].items():
        preset_line = []
        preset_line.append(sg.Button(preset_name,button_color="black on SkyBlue1",key=f'preset_{preset_name}',s=(12,3)))
        for light_source_name, light_source_state in preset_state.items():
            if 'light' in light_source_name:
                hex_color = sg.rgb(*light_source_state[RED_ID-1:BLUE_ID])
                preset_line.append(sg.Text('',background_color=hex_color,s=(12,1)))
        layout.append(preset_line.copy())
    return layout

In [20]:
def create_edit_layout() -> list:
    """ Create the user interface layout using PySimpleGui. """
    layout = [
    [sg.Column([[sg.Text("Control Center Group 1",justification="center",s=(56,1))],
                [sg.Button("Main",button_color="black on SkyBlue1",key="group_1",s=(12,5)),
                 sg.Text('',background_color='white',s=(0,5)),
                 sg.Button("Light 1",button_color="black on SkyBlue1",key="light_1",s=(12,5)),
                 sg.Button("Light 3",button_color="black on SkyBlue1",key="light_3",s=(12,5)),
                 sg.Button("Light 5",button_color="black on SkyBlue1",key="light_5",s=(12,5))],
                [sg.Text("Control Center Group 2",justification="center",s=(56,1))],
                [sg.Button("Ambiance",button_color="black on gold",key="group_2",s=(12,5)),
                 sg.Text('',background_color='white',s=(0,5)),
                 sg.Button("Light 2",button_color="black on gold",key="light_2",s=(12,5)),
                 sg.Button("Light 4",button_color="black on gold",key="light_4",s=(12,5)),
                 sg.Button("Light 6",button_color="black on gold",key="light_6",s=(12,5))],]),
     sg.Column([[sg.Text('Save/Load Controls',justification='center',s=(30,1))],
                [sg.Text('',s=(8,0)),
                 sg.Button("Load Preset",button_color="black on SkyBlue1",key="load",s=(12,5)),
                 sg.Text('',s=(8,0))],
                [sg.Text('',s=(0,1))],
                [sg.Text('',s=(8,0)),
                 sg.Button("Save",button_color="black on SkyBlue1",key="save",s=(12,5)),
                 sg.Text('',s=(8,0))]])],
    [sg.Text("Light Source Under Control: ",justification='center',size=91,key='target')],
    [sg.Column([[sg.Image('../img/color_wheel.png',size=(512,512), enable_events=True, key='color_wheel')]]),
     sg.Column([[sg.Text('Red', s=(6,1)),sg.Slider((0,255), default_value=0, resolution=1, enable_events=True,
                         orientation='horizontal', tick_interval=255, key='slider_red')],
                [sg.Text('Green', s=(6,1)),sg.Slider((0,255), default_value=0, resolution=1, enable_events=True,
                         orientation='horizontal', tick_interval=255, key='slider_green')],
                [sg.Text('Blue', s=(6,1)),sg.Slider((0,255), default_value=0, resolution=1, enable_events=True,
                         orientation='horizontal', tick_interval=255, key='slider_blue')],
                [sg.Text('White', s=(6,1)),sg.Slider((0,255), default_value=0, resolution=1, enable_events=True,
                         orientation='horizontal', tick_interval=255, key='slider_white')],
                [sg.Text('Amber', s=(6,1)),sg.Slider((0,255), default_value=0, resolution=1, enable_events=True,
                         orientation='horizontal', tick_interval=255, key='slider_amber')],
                [sg.Text('UV', s=(6,1)),sg.Slider((0,255), default_value=0, resolution=1, enable_events=True,
                         orientation='horizontal', tick_interval=255, key='slider_uv')],
                [sg.Text('Dimmer', s=(6,1)),sg.Slider((0,255), default_value=0, resolution=1, enable_events=True,
                         orientation='horizontal', tick_interval=255, key='slider_dimmer')]])]]
    return layout

In [21]:
def create_UI_layout(presets:dict) -> list:
    """ Create the user interface layout using PySimpleGui. """
    layout = [[sg.TabGroup([[
                sg.Tab('Presets',create_preset_layout(presets)),
                sg.Tab('Edit',create_edit_layout())]])]]
    return layout

##### Helpers

In [22]:
def update_sliders(window:sg.Window, light_object:LightSource):
    """ Update the sliders with the light source state. """
    for name in list(LIGHT_FIXTURE_EVENTS):
        fixture_id = FIXTURE_TO_ID_DICT[name.split('_')[-1]]
        window[name].update(light_object.state[fixture_id-1])

In [23]:
def update_button(window:sg.Window, light_object:LightSource):
    """ Update the button with the light source state. """
    button_id = light_object.name
    hex_color = sg.rgb(*light_object.state[RED_ID-1:BLUE_ID])
    window[button_id].update(button_color=hex_color)

In [24]:
def update_buttons(window:sg.Window, light_object_dict:dict):
    """ Update all buttons with the light sources state. """
    for light_object in light_object_dict.values():
        update_button(window,light_object)

In [25]:
def save_presets(presets:dict,presets_path:str):
    """ Save the presets given as a dictionnary in json format. """
    with open(presets_path,'w') as file:
        json.dump(presets,file)

##### Process

In [26]:
def preset_process(presets:dict,preset_group:str,light_object_dict:dict):
    """ """
    layout = create_preset_selector_layout(presets,preset_group)
    preset_window = sg.Window('Preset', layout, background_color='black', resizable=False).finalize()
    preset_selected = False
    while True:
        # Update GUI
        event, values = preset_window.read(timeout=1000)
        if event == sg.WIN_CLOSED:
            break 
        elif event != '__TIMEOUT__':
            preset_state = presets[preset_group][event.split('preset_')[-1]]
            for light_source_name, state in preset_state.items():
                light_object_dict[light_source_name].set_fixture_values(state)
                light_object_dict[light_source_name].turn_on()
            preset_selected = True
            break
    preset_window.close();
    return preset_selected

In [27]:
def save_preset_process(presets:dict,light_object_dict:dict,presets_path:str):
    """ """
    layout = create_preset_layout(presets)
    preset_window = sg.Window('Save Preset', layout, background_color='black', resizable=False).finalize()
    while True:
        # Update GUI
        event, values = preset_window.read(timeout=1000)
        if event == sg.WIN_CLOSED:
            break 
        elif event != '__TIMEOUT__':
            text = sg.popup_get_text('Entrez nom du preset', title="Textbox")
            if text != None and text != '':
                if text in presets[event.split(PRESET_BUTTON_PREFIX)[-1]].keys():
                    sg.popup_auto_close('Le nom du preset existe déjà, veuillez en entrer un nouveau.')
                else:
                    presets[event.split(PRESET_BUTTON_PREFIX)[-1]][text] = dict([(name,l.state) for name,l in light_object_dict.items()])
                    save_presets(presets,presets_path)
                    break
    preset_window.close();

In [28]:
def load_preset_process(window:sg.Window,presets:dict,light_object_dict:dict):
    """ """
    layout = create_preset_layout(presets)
    preset_window = sg.Window('Save Preset', layout, background_color='black', resizable=False).finalize()
    while True:
        # Update GUI
        event, values = preset_window.read(timeout=1000)
        if event == sg.WIN_CLOSED:
            break 
        elif event != '__TIMEOUT__':
            preset_selected = preset_process(presets,event.split(PRESET_BUTTON_PREFIX)[-1],light_object_dict)
            if preset_selected:
                update_buttons(window,light_object_dict)
            break
    preset_window.close();

In [29]:
def select_config(presets:dict):
    """ """
    layout = create_preset_layout(presets)
    preset_window = sg.Window('Save Preset', layout, background_color='black', resizable=False).finalize()
    while True:
        # Update GUI
        event, values = preset_window.read(timeout=1000)
        if event == sg.WIN_CLOSED:
            break 
        elif event != '__TIMEOUT__':
            preset_name = event.split(PRESET_BUTTON_PREFIX)[-1]
            bitfocus_config = create_config_structure(DEFAULT_IP,a[preset_name])
            with open(BITFOCUS_CONFIG_FOLDER+preset_name+'.json','w') as f:
                json.dump(bitfocus_config,f)
            break
    preset_window.close();

In [30]:
def UI_process(light_object_dict:dict,presets:dict,presets_path:str):
    """ 
    User interface process, handling user actions. 
    
    :param light_object_dict: Dictionnary with event_id as key and 
                              corresponding light object as value.
    
    """
    layout = create_UI_layout(presets)
    window = sg.Window("Delta Control", layout, background_color='black', resizable=False).finalize()
    window.bind('<Motion>', 'Motion')
    position = pyautogui.position()
    light_object = None
    while True:
        # Update GUI
        event, values = window.read(timeout=1000)
        if event == sg.WIN_CLOSED:
            select_config(presets)
            for light_object in light_object_dict.values():
                light_object.turn_off()
            break
        elif PRESET_BUTTON_PREFIX in event:
            preset_group = event.split(PRESET_BUTTON_PREFIX)[-1]
            preset_selected = preset_process(presets,preset_group,light_object_dict)
            if preset_selected:
                update_buttons(window,light_object_dict)
        elif event == 'save':
            save_preset_process(presets,light_object_dict,presets_path)
        elif event == 'load':
            load_preset_process(window, presets,light_object_dict)
        elif event == 'color_wheel' and light_object != None:
            e = window.user_bind_event
            pixel = ImageGrab.grab(bbox=(
                e.x_root, e.y_root, e.x_root+1, e.y_root+1)).getdata()[0]
            light_object.set_rgb(pixel)
            update_sliders(window,light_object)
            update_button(window,light_object)
        elif event in LIGHT_FIXTURE_EVENTS and light_object != None:
            fixture_id = FIXTURE_TO_ID_DICT[event.split('_')[-1]]
            light_object.set_fixture_value(fixture_id,int(values[event]))
            update_button(window,light_object)
        elif event in LIGHT_SELECTION_EVENTS:
            light_object = light_object_dict[event]
            window['target'].update('Light Source Under Control: '+event)
            update_sliders(window,light_object)
            update_button(window,light_object)
            #light_object.blink()
    window.close();

#### Pipeline

In [31]:
def live_color_picker(ip:str, num_lights:int, groups_mapping=DEFAULT_GROUPS,
                      packet_size=DEFAULT_PACKET_SIZE, fps=DEFAULT_FPS,
                      even_packet_size = ENFORCE_EVEN_PACKET, broadcast=ENFORCE_BROADCAST,
                      universe_id=DEFAULT_UNIVERSE_ID,channel_width=DEFAULT_CHANNEL_WIDTH,
                      presets_path=PRESETS_PATH):
    """
    Pipeline to select color of each light source in real time.

    :param ip: Ip of the ArtNet receiving device.
    :param num_lights: Number of lights to be configured. 
    :param groups_mapping: Mapping between group name and set of lights.
    :param packet_size: Size of ArtNet packets.
    :param fps: Refresh rate of the server.
    :param even_packet_size: Boolean variable to enforce even packets (May be
                             required by the receiver).
    :param broadcast: Boolean variable to allow broadcast in the subnet.
    :param universe_id: Identifier of the universe with which we want to communicate.
    :param channel_width: Number of fixtures per channel.
    :param presets_path: Path to the JSON file containing the presets.
    
    """
    # Init connections
    server = StupidArtnet(ip,universe_id,packet_size,fps,even_packet_size,broadcast)
    with open(PRESETS_PATH,'r') as file:
        presets = json.load(file)
   # Lights
    lights = []
    for i in range(num_lights):
        channel_start = DEFAULT_CHANNEL_START_ID + i*channel_width
        lights.append(Light(name='light_'+str(i+1),channel=Channel(server,channel_start,channel_width)))
    # Groups
    groups = []
    for group_name, group_lights_names in groups_mapping.items():
        group_lights = [l for l in lights if l.name in group_lights_names]
        groups.append(Group(name=group_name, lights=group_lights))
    # light Object Mapping
    light_object_dict = [('group_'+str(i+1),groups[i]) for i in range(len(groups))]
    light_object_dict.extend([('light_'+str(i+1),lights[i]) for i in range(num_lights)])
    light_object_dict = dict(light_object_dict)
    # UI Loop
    UI_process(light_object_dict,presets,presets_path)

### Software

In [32]:
live_color_picker(DEFAULT_IP,6)

---

### Test


In [ ]:
server = StupidArtnet(DEFAULT_IP, 1, 512, 30, True, True)

In [ ]:
c = Channel(server,23,11)

In [ ]:
l = Light('test',c)

In [ ]:
l.set_rgb([255,0,255])

In [ ]:
l.turn_on()

In [ ]:
l.turn_off()

In [ ]:
l.blink(n_repeat=2)

### Archive

#### Config

In [ ]:
def create_button_config(light_source_dict:dict,instance_id:str,preset_name:str,
                        id_length:int,fade_time:int=50) -> dict:
    """ """
    light_dict = dict([(light_name, light_object) for light_name,light_object in 
                        light_source_dict.items() if 'group' not in light_name])
    button_config = {"type":"button",
                     "style":{"text":preset_name,"size":"auto","png":Null,
                              "alignment":"center:top","pngalignment":"center:center",
                              "color":16777215,"bgcolor":0,"show_topbar":True},
                     "options":{"relativeDelay":False,"stepAutoProgress":True},
                     "feedbacks":[],
                     "steps":{"0":{"action_sets":{"down":[
                         create_channel_config(light_object,i+1,instance_id,id_length,fade_time) for
                         i in range(len(light_object.state)) for light_object
                         in light_dict.values()],
                     "up":[]},"options":{"runWhileHeld":[]}}}}
    return button_config
    

In [ ]:
def create_channel_config(light_object:LightSource,fixture_id:int,instance_id:str,id_length:int,fade_time:int) -> dict:
    """ Create config step for a single channel. """
    channel_dict = {"id":generate_config_id(id_length),"action":"set",
                    "instance":instance_id,
                    "options":{"channel":light_object.channel.offset+fixture_id,"value":light_object.state[fixture_id-1],
                    "duration":fade_time},"delay":0}
    return channel_dict

#### Color Picker

In [ ]:
def color_popup(light_object:LightSource):
    """
    Popup window to select the color of the given light object.

    :param light_object: LightSource which will be controlled.
    
    """
    SCREEN = pygame.display.set_mode((400, 400))
    SCREEN.fill(pygame.Color(0, 0, 0))
    pygame.display.init()
    pygame.display.update()
    pygame.init()
    pygame.display.set_caption('Colour Picking App')

    
    ui_manager = pygame_gui.UIManager((400, 400))
    colour_picker = None                                    
    current_colour = pygame.Color(0, 0, 0)
    
    clock = pygame.time.Clock()
    begin_loop = True
    
    while True:
        time_delta = clock.tick(60) / 1000
        event_list = pygame.event.get()
        if begin_loop == True:
            event_list.append(Event(1,{'state':1,'gain':1}))
        for event in event_list:
            if begin_loop == True:
                colour_picker = UIColourPickerDialog(pygame.Rect(0, 0, 400, 400),
                                                    ui_manager,
                                                    window_title=f"Change Colour of {light_object.name}",
                                                    initial_colour=current_colour,
                                                    light_object=light_object)
                begin_loop = False
            if (event.type == pygame.QUIT or event.type == pygame_gui.UI_COLOUR_PICKER_COLOUR_PICKED
                or (event.type == 1 and event.gain == 0)):
                pygame.quit()
                time.sleep(1)
                return
            ui_manager.process_events(event)
            
        ui_manager.update(time_delta)
        ui_manager.draw_ui(SCREEN)
        pygame.display.update()
    
    pygame.quit()

In [ ]:
def color_popup(light_object:LightSource):
    """
    Popup window to select the color of the given light object.

    :param light_object: LightSource which will be controlled.
    
    """
    SCREEN = pygame.display.set_mode((400, 400))
    SCREEN.fill(pygame.Color(0, 0, 0))
    pygame.display.init()
    pygame.display.update()
    pygame.init()

    pygame.display.set_caption('Colour Picking App')
    
    
    ui_manager = pygame_gui.UIManager((400, 400))
    #background = pygame.Surface((400, 400))
    #background.fill("#3a3b3c")
    """
    colour_picker_button = UIButton(relative_rect=pygame.Rect(-180, -60, 150, 30),
                                    text='Pick Colour',
                                    manager=ui_manager,
                                    anchors={'left': 'right',
                                            'right': 'right',
                                            'top': 'bottom',
                                            'bottom': 'bottom'})

    """
    colour_picker = None                                    
    current_colour = pygame.Color(0, 0, 0)
    #picked_colour_surface = pygame.Surface((400, 400))
    #picked_colour_surface.fill(current_colour)
    
    clock = pygame.time.Clock()
    begin_loop = True

    while True:
        time_delta = clock.tick(60) / 1000
        event_list = pygame.event.get()
        if begin_loop == True:
            event_list.append(Event(1,{'state':1,'gain':1}))
        for event in event_list:
            if begin_loop == True:
                colour_picker = UIColourPickerDialog(pygame.Rect(0, 0, 400, 400),
                                                    ui_manager,
                                                    window_title="Change Colour...",
                                                    initial_colour=current_colour,
                                                    light_object=light_object)
                #colour_picker_button.disable()
                begin_loop = False
            if event.type == pygame.QUIT:
                pygame.quit()
                break
            """
            if event.type == pygame_gui.UI_BUTTON_PRESSED and event.ui_element == colour_picker_button:
                colour_picker = UIColourPickerDialog(pygame.Rect(0, 0, 400, 400),
                                                    ui_manager,
                                                    window_title="Change Colour...",
                                                    initial_colour=current_colour,
                                                    light_object=light_object)
                colour_picker_button.disable()
            """
            if event.type == pygame_gui.UI_COLOUR_PICKER_COLOUR_PICKED:
                pygame.quit()
                return
                #current_colour = event.colour
                #picked_colour_surface.fill(current_colour)
            if event.type == 1:
                if event.gain == 0:
                    pygame.quit()
                    return
            if event.type == pygame_gui.UI_WINDOW_CLOSE:
                #colour_picker_button.enable()
                colour_picker = None
            
            ui_manager.process_events(event)
            
        ui_manager.update(time_delta)
    
        #SCREEN.blit(background, (0, 0))
        #SCREEN.blit(picked_colour_surface, (200, 100))
    
        ui_manager.draw_ui(SCREEN)
    
        pygame.display.update()
    
    pygame.quit()